In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from numpy import e


In [39]:
# Set path Selenium
CHROMEDRIVER_PATH = '/usr/local/bin/chromedriver'

In [85]:


def job_posts(number):
    driver = webdriver.Chrome()
    links = []
    position_names = []
    companies = []
    locations = []  
    
    path1 = f'https://www.linkedin.com/jobs/search?keywords=Data%20Scientist&location=United%20States'
    driver.get(path1)
    
    i = 0
    
    while i < number:
        html = driver.find_element(By.TAG_NAME,'html')
        html.send_keys(Keys.END)
        i = i+1
        try:
            t = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/button')
            t.click()
            time.sleep(2)
        except:
            pass
            time.sleep(3)

            
    
    block = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[2]/ul')
    job_list = block.find_elements(By.CLASS_NAME, 'base-card')
    print(len(job_list))
    
    try:

        for job in job_list:
            try:
                link = job.find_element(By.CLASS_NAME, 'base-card__full-link').get_attribute('href')
                links.append(link)
            except:
                links.append(None)
                ValueError('no link')
            try:
                position_name = job.find_element(By.TAG_NAME,'span').text
                position_names.append(position_name)
            except:
                position_names.append(None)
                ValueError('no pos name')
            try:
                company = job.find_element(By.CLASS_NAME,'hidden-nested-link').text
                companies.append(company)
            except:
                companies.append(None)
                ValueError('no company')
            try:
                location = job.find_element(By.CLASS_NAME,'job-search-card__location').text
                locations.append(location)
            except:
                locations.append(None)
                ValueError('no location')

    except:
        e = 'no path'
        ValueError(e)
        print(e)


    data = pd.DataFrame({'Position':position_names,
            'Company':companies,
            'Location':locations,
            'Link':links})

        
    return data

In [95]:
dataframe=job_posts(50)
dataframe.to_csv('/home/jess/Documents/linked_links.csv')

Scraping page number - 0.0
1000


In [ ]:
df = pd.read_csv('/home/jess/Documents/linked_links.csv')
links=df.Link.tolist()

In [ ]:
driver = webdriver.Chrome()
levels = []
linkss =[] 
employment = []
job_functions = []
industries = []
contents = []

for link in links:
    try:
        driver.get(link)
        driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
        time.sleep(1)     
        try:
            linkss.append(link)
        except:
            linkss.append(None)
            ValueError('No link')

        try:
            Seniority_Level = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[1]/span').text
            levels.append(Seniority_Level)
        except:
            levels.append(None)
            ValueError('No level')
        try:
            Employment_Type = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[2]/span').text
            employment.append(Employment_Type)
        except:
            employment.append(None)
            ValueError('no employment')
        
        try:
            Job_Function = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[3]/span').text
            job_functions.append(Job_Function)
        except:
            job_functions.append(None)
            ValueError('no job function')
        
        try:
            Industry = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div[1]/section[1]/div/ul/li[4]/span').text
            industries.append(Industry)
        except:
            industries.append(None)
            ValueError('no industry')     

        try:
            content=driver.find_element(By.CLASS_NAME,'show-more-less-html__markup').text
            contents.append(content)
        except:
            contents.append(None)
            ValueError('no content')     

    except:
        ValueError('smthng is wrong')
    time.sleep(5)

c_data = pd.DataFrame({'links':linkss,
                        'content':contents,
                        'level':levels,
                        'job_function':job_functions,
                        'industries':industries,
                        'employment':employment

})
c_data

In [ ]:
c_data.to_csv('/home/jess/Documents/c_data.csv')

df_merged = c_data.merge(dataframe,how='left',right_on='Link',left_on='links')
df_merged = df_merged.drop(columns=['Link','links','Unnamed: 0_x','Unnamed: 0_y'])